<a href="https://colab.research.google.com/github/Vishesh-Alag/Airbnb-DataViz-Dashboard-PowerBI/blob/main/Python_Airbnb_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load both CSVs
chicago_df = pd.read_csv('airbnb_chicago_listings.csv')
nola_df = pd.read_csv('airbnb_new_orleans_listings.csv')

# Define cleaning function
def clean_airbnb_data(df, city_name):
    # Standardize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

    # Add city column
    df['city'] = city_name

    # Convert 'last_review' to datetime
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

    # Drop duplicates
    df = df.drop_duplicates()

    # Strip whitespace from string columns
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip()

    # Clean neighbourhood text
    if 'neighbourhood' in df.columns:
        df['neighbourhood'] = df['neighbourhood'].str.title()

    # Drop rows with missing critical values
    df = df.dropna(subset=['price', 'last_review', 'reviews_per_month'])

    # Convert latitude and longitude to float
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

    # Remove rows with missing lat/lon
    df = df.dropna(subset=['latitude', 'longitude'])

    return df

# Clean datasets
chicago_clean = clean_airbnb_data(chicago_df, 'Chicago')
nola_clean = clean_airbnb_data(nola_df, 'New Orleans')

# Combine them
combined_df = pd.concat([chicago_clean, nola_clean], ignore_index=True)

# Feature Engineering

# Host type
combined_df['host_type'] = combined_df['calculated_host_listings_count'].apply(
    lambda x: 'Multi-Listing Host' if x > 1 else 'Single-Listing Host'
)

# Price category
combined_df['price_category'] = combined_df['price'].apply(
    lambda x: 'Low' if x < 100 else 'Medium' if x <= 200 else 'High'
)

# Reviewed recently (within last 180 days)
cutoff_date = pd.to_datetime(datetime.today() - timedelta(days=180))
combined_df['reviewed_recently'] = combined_df['last_review'].apply(
    lambda x: 'Yes' if pd.notnull(x) and x >= cutoff_date else 'No'
)

# Replace any remaining NaNs with blanks for Power BI
combined_df.fillna('', inplace=True)

# Reorder columns (optional, for better structure in Power BI)
column_order = [
    'name', 'city', 'neighbourhood', 'latitude', 'longitude', 'room_type',
    'price', 'minimum_nights', 'number_of_reviews', 'last_review',
    'reviews_per_month', 'calculated_host_listings_count', 'availability_365',
    'number_of_reviews_ltm', 'host_type', 'price_category', 'reviewed_recently'
]
combined_df = combined_df[[col for col in column_order if col in combined_df.columns]]

# Save cleaned CSV with UTF-8 encoding
combined_df.to_csv('cleaned_airbnb_combined.csv', index=False, encoding='utf-8')

print("✅ Cleaned data saved as cleaned_airbnb_combined.csv")


✅ Cleaned data saved as cleaned_airbnb_combined.csv


C:\Users\ASUS\AppData\Local\Temp\ipykernel_18192\3317718849.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined_df.fillna('', inplace=True)
